In [1]:
import os
import numpy as np
import pandas as pd
from lets_plot import *
from lets_plot import ggplot, aes, geom_histogram, geom_density, facet_wrap, labs, ggsize, ggsave

LetsPlot.setup_html()

# Paths
preprocessed_data_path = '../../data/HOORRAAH_final_banking_indicators_preprocessed.parquet'
output_dir_html = '../../visualisations/html'
os.makedirs(output_dir_html, exist_ok=True)

if not os.path.exists(preprocessed_data_path):
    print(f"Error: Preprocessed data file not found at {preprocessed_data_path}")
else:
    print(f"Loading preprocessed data from {preprocessed_data_path}...")
    df = pd.read_parquet(preprocessed_data_path)
    n_rows = len(df)
    print("Rows loaded:", n_rows)

    preferred = [
        'ROA', 'ROE', 'NIM', 'npl_ratio', 'llp_to_loans_ratio',
        'coverage_ratio', 'loan_to_deposit_ratio', 'log_total_assets',
        'total_assets', 'total_loans', 'total_deposits', 'net_income_amount'
    ]

    # Force numeric where possible (Parquet decimals can arrive as 'object')
    for c in df.columns:
        if pd.api.types.is_numeric_dtype(df[c]) or df[c].dtype == "object":
            df[c] = pd.to_numeric(df[c], errors='coerce')

    numeric_columns = df.select_dtypes(include=[np.number]).columns.tolist()

    selected = [c for c in preferred if c in numeric_columns]
    max_vars = 12
    if len(selected) < max_vars:
        for c in numeric_columns:
            if c not in selected:
                selected.append(c)
            if len(selected) >= max_vars:
                break

    print("Selected variables for grid plots:", selected)


Loading preprocessed data from ../../data/HOORRAAH_final_banking_indicators_preprocessed.parquet...
Rows loaded: 178758
Selected variables for grid plots: ['ROA', 'ROE', 'NIM', 'npl_ratio', 'llp_to_loans_ratio', 'coverage_ratio', 'loan_to_deposit_ratio', 'log_total_assets', 'total_assets', 'total_loans', 'total_deposits', 'net_income_amount']


In [2]:

id_cols = ['DT', 'REGN', 'form']
to_keep = [c for c in id_cols if c in df.columns] + selected
df_sub = df[to_keep].copy()

# Build a long frame and make sure value is numeric float
melted = df_sub.melt(
    id_vars=[c for c in id_cols if c in df_sub.columns],
    value_vars=selected,
    var_name='variable',
    value_name='value'
)
melted['value'] = pd.to_numeric(melted['value'], errors='coerce')
melted = melted.replace([np.inf, -np.inf], np.nan).dropna(subset=['value'])

# --- Robust per-variable clipping (guards against all-NaN or constant cols) ---
low_q, high_q = 0.005, 0.995
bounds = {}
for v in selected:
    s = pd.to_numeric(df[v], errors='coerce')
    s = s.replace([np.inf, -np.inf], np.nan).dropna()
    if s.size == 0:
        continue
    lo = np.nanquantile(s, low_q)
    hi = np.nanquantile(s, high_q)
    # Fallbacks if quantiles are weird
    if not np.isfinite(lo) or not np.isfinite(hi) or lo == hi:
        lo, hi = np.nanmin(s), np.nanmax(s)
    if np.isfinite(lo) and np.isfinite(hi):
        bounds[v] = (lo, hi)

# Keep only variables with usable bounds
melted = melted[melted['variable'].isin(bounds.keys())].copy()
melted['lo'] = melted['variable'].map({k: v[0] for k, v in bounds.items()})
melted['hi'] = melted['variable'].map({k: v[1] for k, v in bounds.items()})
mask = (melted['value'] >= melted['lo']) & (melted['value'] <= melted['hi'])
melted_clip = melted[mask].copy()

# Drop variables that ended up empty post-clipping (prevents blank facets)
cnts = melted_clip['variable'].value_counts()
keep_vars = cnts.index.tolist()
melted_clip = melted_clip[melted_clip['variable'].isin(keep_vars)].copy()

print("Rows before clipping:", len(melted), "Rows after clipping:", len(melted_clip))
print("Variables retained:", sorted(set(melted_clip['variable'])))

Rows before clipping: 2145094 Rows after clipping: 2127006
Variables retained: ['NIM', 'ROA', 'ROE', 'coverage_ratio', 'llp_to_loans_ratio', 'loan_to_deposit_ratio', 'log_total_assets', 'net_income_amount', 'npl_ratio', 'total_assets', 'total_deposits', 'total_loans']


In [3]:
# ---------- Add a viz-friendly transform per variable ----------
def is_non_negative(s: pd.Series) -> bool:
    s = pd.to_numeric(s, errors='coerce')
    return (s.dropna() >= 0).all()

# Decide transform for each variable
transform_map = {}
for v in sorted(set(melted_clip['variable'])):
    s = pd.to_numeric(df[v], errors='coerce').replace([np.inf, -np.inf], np.nan).dropna()
    if s.empty:
        continue
    if is_non_negative(s):
        transform_map[v] = 'log1p' if (s.max() / (s.median() + 1e-12) > 100) else 'linear'
    else:
        transform_map[v] = 'signed_log' if (s.abs().max() / (s.abs().median() + 1e-12) > 100) else 'linear'

# Build a transformed copy for plotting
melted_viz = melted_clip.copy()
def apply_transform(val, t):
    if t == 'log1p':
        return np.log1p(np.maximum(val, 0))
    if t == 'signed_log':
        return np.sign(val) * np.log1p(np.abs(val))
    return val

melted_viz['transform'] = melted_viz['variable'].map(transform_map)
melted_viz['value_viz'] = [
    apply_transform(v, t) for v, t in zip(melted_viz['value'], melted_viz['transform'])
]
melted_viz['facet'] = melted_viz['variable'] + ' [' + melted_viz['transform'] + ']'
print("Viz transforms used:", {k: transform_map[k] for k in sorted(transform_map)})


Viz transforms used: {'NIM': 'signed_log', 'ROA': 'signed_log', 'ROE': 'signed_log', 'coverage_ratio': 'signed_log', 'llp_to_loans_ratio': 'signed_log', 'loan_to_deposit_ratio': 'signed_log', 'log_total_assets': 'linear', 'net_income_amount': 'signed_log', 'npl_ratio': 'signed_log', 'total_assets': 'signed_log', 'total_deposits': 'signed_log', 'total_loans': 'signed_log'}


In [4]:
# Pretty labels with LaTeX (Lets-Plot renders these directly)
pretty_labels = {
    'NIM': r'NIM',
    'ROA': r'ROA',
    'ROE': r'ROE',
    'coverage_ratio': r'Coverage ratio',
    'llp_to_loans_ratio': r'LLP ÷ Loans',
    'loan_to_deposit_ratio': r'Loans ÷ Deposits',
    'log_total_assets': r'log(Total assets)',
    'total_assets': r'Total assets',
    'total_deposits': r'Total deposits',
    'total_loans': r'Total loans',
    'net_income_amount': r'Net income',
    'npl_ratio': r'NPL ratio',
}

melted_viz = melted_viz.copy()
melted_viz['pretty'] = melted_viz['variable'].map(pretty_labels).fillna(melted_viz['variable'])
melted_viz['facet']  = melted_viz['pretty'] + '\n[' + melted_viz['transform'] + ']'


In [5]:
from lets_plot import sampling_random_stratified, sampling_systematic, layer_tooltips, theme

# --- Sampling budget ---
N_TOTAL_BUDGET = 300_000              # cap total rows used by each layer
n_facets = melted_viz['facet'].nunique()
N_PER_FACET  = max(2_000, N_TOTAL_BUDGET // n_facets)

# Stratify by facet so every variable is represented,
# then add a "safety" systematic sampler as a final cap.
sampler = sampling_random_stratified(N_PER_FACET, min_subsample=1_000) + sampling_systematic(N_TOTAL_BUDGET)

# We'll also disable tooltips to shrink HTML size:
tt_none = 'none'   # same as layer_tooltips(...); use the string 'none' to hide.  # noqa


In [6]:
# Order facets by IQR on the viz scale so the "most informative" panels come first.
iqr = (melted_viz.groupby('facet')['value_viz']
                  .quantile([0.75, 0.25]).unstack())
facet_order = (iqr[0.75] - iqr[0.25]).sort_values(ascending=False).index.tolist()
melted_viz['facet'] = pd.Categorical(melted_viz['facet'], categories=facet_order, ordered=True)


In [17]:
p_density_viz_sampled = (
    ggplot(melted_viz, aes(x='value_viz', group='facet'))
    + geom_density(
        sampling=sampler,
        n=256,            # fewer eval points -> faster/lighter
        trim=True,        # ignore far tails (we’ve already winsorised 0.5–99.5%)
        quantiles=[0.25, 0.5, 0.75],
        quantile_lines=True,
        tooltips=tt_none
      )
    + facet_wrap('facet', scales='free')
    + labs(title='Grid: Density (viz transform + sampling + quartiles)',
           x='Value (viz scale)', y='Density')
    + theme(exponent_format='pow')
    + ggsize(1400, 900)
)
# ggsave(p_density_viz_sampled, path=output_dir_html, filename='variable_grid_density_viz_sampled.html')
p_density_viz_sampled

In [70]:
import numpy as np
import pandas as pd

def _pct(x): 
    return 100.0 * (x.sum() / max(len(x), 1))

g = melted_viz.groupby('facet')['value_viz']

stats = pd.DataFrame({
    'facet': g.size().index,
    'n': g.size().values,
    'zeros_pct': g.apply(lambda s: (s == 0).mean() * 100.0).values,
    'q05': g.quantile(0.05).values,
    'q25': g.quantile(0.25).values,
    'q50': g.quantile(0.50).values,
    'q75': g.quantile(0.75).values,
    'q95': g.quantile(0.95).values,
})

stats['iqr'] = stats['q75'] - stats['q25']
stats['tail_ratio'] = (stats['q95'] - stats['q50']) / (stats['q50'] - stats['q05'] + 1e-12)

# Position the label near the upper-left of each panel
# x: left edge (5th pct). y: rough density peak using robust sigma.
robust_sigma = np.clip(stats['iqr'] / 1.349, 1e-6, None)
y_est = 1.0 / (np.sqrt(2*np.pi) * robust_sigma)               # ~ Normal peak
y_cap = np.quantile(y_est, 0.85)                               # keep labels inside view
stats['x_pos'] = stats['q05']
stats['y_pos'] = np.minimum(y_est, y_cap) * 0.9

# Multi-line card text
stats['label'] = (
    'n={n:,}\nzeros={zeros:.1f}%\nmedian={med:.3g}\nIQR={iqr:.3g}\nTailR={tr:.2f}'
    .format  # use vectorized formatting via apply below
)
stats['label'] = stats.apply(
    lambda r: f"n={int(r['n']):,}\nzeros={r['zeros_pct']:.1f}%\nmedian={r['q50']:.3g}\nIQR={r['iqr']:.3g}\nTailR={r['tail_ratio']:.2f}",
    axis=1
)


/var/folders/tk/3rj0dqn167qd5hxxfjtpq6q00000gn/T/ipykernel_46936/1669490143.py:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  g = melted_viz.groupby('facet')['value_viz']


In [7]:
from lets_plot import theme, element_text

SMALL = theme(
    plot_title   = element_text(size=16),
    axis_title   = element_text(size=10),
    axis_text_x  = element_text(size=8),
    axis_text_y  = element_text(size=8),
    strip_text   = element_text(size=9),
    legend_text  = element_text(size=8),
    legend_title = element_text(size=9),
    exponent_format='pow',
)


In [8]:
import numpy as np
import pandas as pd

g = melted_viz.groupby('facet')['value_viz']

q = g.quantile([0.05, 0.10, 0.25, 0.50, 0.75, 0.90, 0.95]).unstack()
q.columns = ['q05','q10','q25','q50','q75','q90','q95']

stats = pd.DataFrame({
    'facet': q.index,
    'n': g.size().values,
    'zeros_pct': g.apply(lambda s: (s == 0).mean() * 100.0).values,
    'q05': q['q05'].values, 'q10': q['q10'].values,
    'q25': q['q25'].values, 'q50': q['q50'].values, 'q75': q['q75'].values,
    'q90': q['q90'].values, 'q95': q['q95'].values
})
stats['iqr'] = stats['q75'] - stats['q25']

# Choose the sparser side near the edges: left [5–10%] vs right [90–95%]
def sparse_side(s, q05, q10, q90, q95):
    left  = s.between(q05, q10).sum() / max(q10 - q05, 1e-9)
    right = s.between(q90, q95).sum() / max(q95 - q90, 1e-9)
    return 'left' if left <= right else 'right'

side_map = {}
for f, s in g:
    r = stats.loc[stats['facet'].eq(f)].iloc[0]
    side_map[f] = sparse_side(s, r['q05'], r['q10'], r['q90'], r['q95'])

stats['side'] = stats['facet'].map(side_map)

# Horizontal position: just outside the data on the chosen side
stats['x_range'] = stats['q95'] - stats['q05']
stats['x_pad']   = 0.15 * stats['x_range']
stats['x_pos']   = np.where(stats['side'].eq('left'),
                            stats['q05'] - stats['x_pad'],
                            stats['q95'] + stats['x_pad'])

# Vertical position: slightly above a robust peak estimate
robust_sigma = np.clip(stats['iqr'] / 1.349, 1e-6, None)
peak_est     = 1.0 / (np.sqrt(2*np.pi) * robust_sigma)
stats['y_pos'] = peak_est * 1.10

# Formatted multi-line card
stats['label'] = stats.apply(
    lambda r: f"n={int(r['n']):,}\nzeros={r['zeros_pct']:.1f}%\nmedian={r['q50']:.3g}\nIQR={r['iqr']:.3g}",
    axis=1
)

# Split into two data frames to control hjust per side
stats_left  = stats[stats['side'].eq('left')].copy()
stats_right = stats[stats['side'].eq('right')].copy()


/var/folders/tk/3rj0dqn167qd5hxxfjtpq6q00000gn/T/ipykernel_57370/3418362712.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  g = melted_viz.groupby('facet')['value_viz']


In [9]:
# ---- Absolute top-right (~85%, ~85%) per facet ----
stats_tr = stats.copy()

# X: 85% across the visible range (use winsor bounds you already computed)
stats_tr['x_pos'] = stats_tr['q05'] + 0.85 * (stats_tr['q95'] - stats_tr['q05'])

# Y: 85% of an easy per-facet "y max" proxy from robust sigma
robust_sigma = np.clip(stats_tr['iqr'] / 1.349, 1e-6, None)
y_peak = 1.0 / (np.sqrt(2*np.pi) * robust_sigma)   # ~mode density height
y_max_est = y_peak * 1.6                          # small headroom
stats_tr['y_pos'] = 0.93 * y_max_est

# (If any panels look tight, bump the multipliers a touch, e.g. 0.88 / 1.35)


In [13]:
from lets_plot import (
    ggplot, aes, geom_density, geom_label, facet_wrap,
    labs, ggsize, scale_x_continuous, scale_y_continuous
)

p_density_viz_topright = (
    ggplot(melted_viz, aes(x='value_viz', group='facet'))
    + geom_density(
        sampling=sampler,
        n=256, trim=True,
        quantiles=[0.25, 0.5, 0.75],
        quantile_lines=True,
        tooltips='none'
      )
    + geom_label(
        aes(x='x_pos', y='y_pos', label='label'),
        data=stats_tr,
        hjust=16, vjust=1,
        size=3, alpha=0.10, fill='gray92',
        nudge_x=0.3
      )
    + facet_wrap('facet', scales='free', labwidth=24)   # documented API, no tilde
    + scale_x_continuous(expand=[0.06, 0])             # a bit of breathing room
    + scale_y_continuous(expand=[0.10, 0])
    + labs(
        # title='Grid: Density (viz transform + sampling + quartiles + stats @ top-right)',
           x='Value (viz scale)', y='Density')
    + SMALL
    + ggsize(1400, 900)
)

p_density_viz_topright


In [14]:
ggsave(p_density_viz_topright, path=output_dir_html, filename='variable_grid_density_viz_topright_stats.html')
output_dir_svg = '../../visualisations/svg'
os.makedirs(output_dir_svg, exist_ok=True)
ggsave(p_density_viz_topright, path=output_dir_svg, filename='variable_grid_density_viz_topright_stats.svg')

'/Users/alexandersoldatkin/projects/russian_bank_accounting/visualisations/svg/variable_grid_density_viz_topright_stats.svg'